In [1]:
model_name = 'baseline_sparse'
data_dir = './data/mlboot_dataset/'
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import scipy.sparse as sp

In [2]:
df = pd.read_csv(data_dir + 'preprocessed.csv') 
q = pd.read_csv(data_dir + 'sessions.csv')
df = df.merge(q, on='uid', how='left')
del q
y = pd.read_table(data_dir + 'mlboot_train_answers.tsv')
y.columns = ['uid','target']
df = df.merge(y, on='uid', how='left')

mat1 = sp.load_npz(data_dir+'dmat1.npz').tolil()
mat2 = sp.load_npz(data_dir+'dmat2.npz').tolil()
mat3 = sp.load_npz(data_dir+'dmat3.npz').tolil()

In [3]:
df_train_index = df[~df.target.isnull()].index
df_test_index = df[df.target.isnull()].index

X = df.loc[~df.target.isnull(),:].reset_index(drop=True)
x_te = df.loc[df.target.isnull(),:].reset_index(drop=True)

In [4]:
%%time
train_mat1 = mat1[df_train_index.tolist()]

CPU times: user 6.04 s, sys: 220 ms, total: 6.26 s
Wall time: 5.69 s


In [5]:
test_mat1 = mat1[df_test_index.tolist()]
train_mat2 = mat2[df_train_index.tolist()]
test_mat2 = mat2[df_test_index.tolist()]
train_mat3 = mat3[df_train_index.tolist()]
test_mat3 = mat3[df_test_index.tolist()]
limit = 0
mat1 = mat1.tocsc()[:, np.where((train_mat1.getnnz(axis=0) > limit) & (test_mat1.getnnz(axis=0) > limit))[0]].tocsr()
mat2 = mat2.tocsc()[:, np.where((train_mat2.getnnz(axis=0) > limit) & (test_mat2.getnnz(axis=0) > limit))[0]].tocsr()
mat3 = mat3.tocsc()[:, np.where((train_mat3.getnnz(axis=0) > limit) & (test_mat3.getnnz(axis=0) > limit))[0]].tocsr()
train_mat1 = mat1[df_train_index.tolist()]
test_mat1 = mat1[df_test_index.tolist()]
train_mat2 = mat2[df_train_index.tolist()]
test_mat2 = mat2[df_test_index.tolist()]
train_mat3 = mat3[df_train_index.tolist()]
test_mat3 = mat3[df_test_index.tolist()]

In [6]:
train_mat = sp.hstack([train_mat1,train_mat2,train_mat3]).tocsr()
test_mat = sp.hstack([test_mat1,test_mat2,test_mat3]).tocsr()

In [7]:
pca_mat = np.load(data_dir + 'pca_cat100.npy')
train_pca_mat = pca_mat[df_train_index.tolist()]
test_pca_mat = pca_mat[df_test_index.tolist()]

In [ ]:
from sklearn.feature_selection import mutual_info_classif,chi2
from sklearn.feature_selection import SelectPercentile
train_cols = [
       'sess_keys_mean','sess_keys_max','diff_key1_mean','diff_key1_max','diff_key2_mean',
       'diff_key2_max','diff_key3_mean','diff_key3_max','quot_key1_mean','quot_key1_max',
       'quot_key2_mean','quot_key2_max','quot_key3_mean','quot_key3_max',
       u'num_keys_f1',
       u'sum_values_f1', u'num_keys_f2', u'sum_values_f2', u'num_keys_f3',
       u'sum_values_f3', u'num_times_cat_eq_0', u'num_times_cat_eq_1',
       u'num_times_cat_eq_2', u'num_times_cat_eq_3', u'num_times_cat_eq_4',
       u'num_times_cat_eq_5', u'records', u'max_days', u'min_days',
       u'sum_values_f1_std', u'num_keys_f1_std', u'sum_values_f2_std',
       u'num_keys_f2_std', u'sum_values_f3_std', u'num_keys_f3_std',
       u'sum_values_f1_max', u'num_keys_f1_max', u'sum_values_f2_max',
       u'num_keys_f2_max', u'sum_values_f3_max', u'num_keys_f3_max',
       u'sum_values_f1_mean', u'num_keys_f1_mean', u'sum_values_f2_mean',
       u'num_keys_f2_mean', u'sum_values_f3_mean', u'num_keys_f3_mean']

# Train the model
parameters = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'num_leaves': 128,
    'max_depth' : 8,
    'lambda_l1' : 10.5,
    'lambda_l2' : 1.2,
    'bagging_freq' : 2,
    'bagging_fraction' : 0.8,
    #'is_unbalance': True,
    'learning_rate': 0.05,
    'feature_fraction': 0.8,
    'verbose': 0
}

kf = KFold(n_splits=5, shuffle=True, random_state=239)

ifold = 0

y_pred = 0
y_oof = X[['uid','target']].copy()
y_oof['target'] = np.nan

scores = []

for train_index,test_index in kf.split(X):    
    if ifold == 0:
        ifold = 1
        continue
    print('fold', ifold)
    
    y_tr,y_va = X.loc[train_index,'target'].values,X.loc[test_index,'target'].values
    X_tr,X_va = X.loc[train_index, train_cols].values,X.loc[test_index, train_cols].values
    X_te = x_te[train_cols]
    
    #mat = test_mat1.sum(axis=0)
    #ixs1 = np.asarray(mat)[0].argsort()[-1000:][::-1]
    #mat = test_mat2.sum(axis=0)
    #ixs2 = np.asarray(mat)[0].argsort()[-20000:][::-1]
    #mat = test_mat3.sum(axis=0)
    #ixs3 = np.asarray(mat)[0].argsort()[-1000:][::-1]
    
    yy = y_tr
    ssp = SelectPercentile(percentile=0.1)   #mutual_info_classif, 
    ssp.fit(train_mat[train_index], yy)
    sp_train_mat = ssp.transform(train_mat[train_index])
    sp_val_mat = ssp.transform(train_mat[test_index])
    sp_test_mat = ssp.transform(test_mat) 
    del ssp
    ssp = SelectPercentile(mutual_info_classif, percentile=80)
    ssp.fit(sp_train_mat, yy)
    sp_train_mat = ssp.transform(sp_train_mat)
    sp_val_mat = ssp.transform(sp_val_mat)
    sp_test_mat = ssp.transform(sp_test_mat) 
    
    print('prepare train')
    X_tr = sp.hstack([
        X_tr, sp_train_mat,train_pca_mat[train_index]
    ]).tocsr()
    print(X_tr.shape)
    print('prepare valid')
    X_va = sp.hstack([
        X_va, sp_val_mat,train_pca_mat[test_index]
    ]).tocsr()
    
    # Create the LightGBM data containers
    tr_data = lgb.Dataset(X_tr, label=y_tr) #, categorical_feature=cate_cols
    va_data = lgb.Dataset(X_va, label=y_va) #, categorical_feature=cate_cols

    model = lgb.train(parameters,
                      tr_data,
                      valid_sets=va_data,
                      num_boost_round=8000,
                      early_stopping_rounds=200,
                      verbose_eval=50)
    
    yhat = model.predict(X_va, model.best_iteration)
    print(ifold,roc_auc_score(y_va,yhat))
    scores.append(roc_auc_score(y_va,yhat))
    y_oof.loc[test_index,'target'] = yhat

    print('prepare test')
    X_te = sp.hstack([
        X_te, sp_test_mat,test_pca_mat
    ]).tocsr()   
    
    ytst = model.predict(X_te, model.best_iteration)
    y_pred += ytst*0.1
    
    ifold += 1

fold 1


/root/miniconda3/lib/python3.6/site-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [0 0 0 ..., 0 0 0] are constant.
  UserWarning)
/root/miniconda3/lib/python3.6/site-packages/sklearn/feature_selection/univariate_selection.py:114: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


In [16]:
print(scores,np.mean(scores))
roc_auc_score(X.target.values, y_oof.target.values)

[0.68292413655060324, 0.67649067087887793, 0.67748300747367651, 0.68149412930654885, 0.67871625837012206] 0.679421640516


0.67935701957848771

In [17]:
model_name = 'sparse_10folds_timp'

results_dir = './results/'
np.save(results_dir + 'train_' + model_name +'.npy', y_oof.target.values)
sample_sub = pd.read_table(data_dir+'mlboot_test.tsv')

In [18]:
sub = x_te[['uid','target']].copy()
sub['target'] = y_pred
sub.columns = ['cuid','target']
sample_sub = sample_sub.merge(sub, on='cuid', how='left')
np.save(results_dir + 'test_' + model_name +'.npy', sample_sub.target.values)
print('isnull?',sample_sub.target.isnull().any())
sample_sub.head()

isnull? False


,cuid,target
0,888b238b4d14c03173baa375a739f6bc,0.056891
1,ac4b8244f3ae82df511b002257473c11,0.030653
2,483d8b91e49522c8a5bbe37f3872c749,0.048096
3,4c7ec46a0e88a7e1e1cedd2d526d5d61,0.018669
4,fdbfba9842ff0bf86d600eb334c7c42b,0.019849


In [19]:
sample_sub[['target']].to_csv(results_dir + model_name + '.csv', header=False, index=False)